In [2]:
import pandas as pd
#A mejorar
file_loc = "..\data_isna.csv"
df = pd.read_csv(file_loc, header= None)

print(df.shape)
df.head()




(13973, 235)


,0,1,2,3,4,5,6,7,8,9,...,225,226,227,228,229,230,231,232,233,234
0,1,1,1,1,0,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [5]:
from gurobipy import *

m = Model()
# Variables 
x = m.addVars(num_filas, lb = 0, ub = 1, vtype = GRB.BINARY, name = 'x')
y = m.addVars(num_cols,  lb = 0, ub = 1, vtype = GRB.BINARY, name = 'y')
z = m.addVars(num_filas, num_cols, lb = 0, ub = 1, obj = 1, name = 'z')
# ¡Ligar variables …!
m.update()
m
#x[0]


ModuleNotFoundError: No module named 'gurobipy'

In [3]:
#Restricciones 

Con1 = m.addConstrs((z[i,j] == x[i] * y[j] for i in fils for j in cols), 'DefZ') 
Con2 = m.addConstrs((z[i,j] <= x[i]  for i in fils for j in cols), 'LimitZ.1')
Con3 = m.addConstrs((z[i,j] <= y[j]  for i in fils for j in cols), 'LimitZ.2')
Con4 = m.addConstrs((x[i] +  y[j] <= 1 + df.iloc[i, j]  for i in fils for j in cols), 'R.Factibilidad' )
Con5 = m.addConstr(y[4] == 1, 'Fixed P/E')
Con6 = m.addConstr(y[47] == 1, 'Fixed Revenue')
Con7 = m.addConstr(y[92] == 1, 'Fixed Current Ratio')
Con8 = m.addConstr(y[164] == 1, 'Fixed P/Book')
Con9 = m.addConstr(y[89] == 1, 'Fixed Long Term Debt')
Con10 = m.addConstr(y[119] == 1, 'Fixed Working Capital')
Con11 = m.addConstr(y[69] == 1, 'Fixed Dividend History')
Con12 = m.addConstr(y[21] == 1, 'Fixed Net Income History')
Con13 = m.addConstr(y[17] == 1, 'Fixed Net Income Growth')




m


<gurobi.Model MIP instance Unnamed: 0 constrs, 3297863 vars, No parameter changes>

In [4]:
#Objective
m.setObjective( z.sum('*', '*'), GRB.MAXIMIZE)

# Results
m.optimize()
opt_sol = m.getAttr('x')
opt_fil = opt_sol[0:num_filas]
opt_col = opt_sol[num_filas: num_cols + num_filas ]

idx_fils = []
idx_cols = []
for i in range(0,num_filas):
    if opt_fil[i] == 1:
        idx_fils.append(i+1)
for j in range(0,num_cols):
    if opt_col[j] == 1:
        idx_cols.append(j+1)

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 9850974 rows, 3297863 columns and 19701939 nonzeros
Model fingerprint: 0xa548477a
Model has 3283655 quadratic constraints
Variable types: 3283655 continuous, 14208 integer (14208 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 0 rows and 0 columns (presolve time = 8s) ...
Presolve removed 2936603 rows and 0 columns (presolve time = 11s) ...
Presolve removed 7302713 rows and 0 columns (presolve time = 16s) ...
Presolve removed 7351718 rows and 49005 columns (presolve time = 20s) ...
Presolve removed 7351718 rows and 49005 columns (presolve time = 25s) ...
Presolve removed 7351718 rows and 49005 columns (presolve time = 30s) ...
Presolve removed 7351718 rows and 49005 columns (presolve time = 35s) ...
Presolve 

In [ ]:
sr_fil = pd.Series(idx_fils)
sr_col = pd.Series(idx_cols)

sr_fil.to_csv("fil_relevantes2.csv", header = False, index = False)
sr_col.to_csv("col_relevantes2.csv", header = False, index = False)


